### 예스 24 크롤링

In [33]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/jiwon/.wdm/drivers/chromedriver/mac64/120.0.6099.109/chromedriver-mac-arm64/chromedriver'

In [34]:
from selenium import webdriver
browser = webdriver.Chrome()


In [44]:
from selenium.webdriver.common.by import By
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)


In [47]:
from selenium.webdriver.common.by import By
element = browser.find_element(By.CLASS_NAME, 'uU7dJb')
element.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".uU7dJb"}
  (Session info: chrome=120.0.6099.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102b184dc chromedriver + 4162780
1   chromedriver                        0x0000000102b10664 chromedriver + 4130404
2   chromedriver                        0x0000000102767bc0 chromedriver + 293824
3   chromedriver                        0x00000001027ae040 chromedriver + 581696
4   chromedriver                        0x00000001027edab0 chromedriver + 842416
5   chromedriver                        0x00000001027a16bc chromedriver + 530108
6   chromedriver                        0x00000001027a2930 chromedriver + 534832
7   chromedriver                        0x0000000102adde08 chromedriver + 3923464
8   chromedriver             

In [48]:
# 1 페이지의 링크 데이터 전부 수집
### 한개의 베스트셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터
# browser.find_element(By.CLASS_NAME, 'gd_name') #element : 요소
datas = browser.find_element(By.CLASS_NAME, 'gd_name') #elements : 리스트

for i in datas:
    print(i.get_attribute('href'))


TypeError: 'WebElement' object is not iterable

In [40]:
# 3 페이지까지의 링크 데이터 전부 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24

import time

link_list = []
for i in range(1,4):
    print("*" * 10, f"현재 {i} 페이지 수집 중 입니다.", "*" * 10)
    url = f"https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24"
    browser.get(url)
 
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크 데이터
    # browser.find_element(By.CLASS_NAME, 'gd_name') #element : 요소
    datas = browser.find_element(By.CLASS_NAME, 'gd_name') #elements : 리스트

    for i in datas:
        link = i.get_attribute('href')

        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********


TypeError: 'WebElement' object is not iterable

In [ ]:
len(link_list)

In [52]:
for link in link_list:
# 상세페이지로 이동
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').text
    author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
    publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
    publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
    rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
    reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
    sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
    price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
    ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]
    ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]

    
    

In [29]:
text = "'| 판매지수 932,772 판매지수란?'"

text.split(" ")[2]

'932,772'

In [32]:
text = "'베스트 국내도서 1위 | 국내도서 1위 5주'"
text.split(" | ")[0]

["'베스트 국내도서 1위", "국내도서 1위 5주'"]

In [56]:
# 데이터베이스 연동 후 -> 수집한 데이터를 DB에 저장(csv)
import pymysql # pip3 install pymysql
import time

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '900326',
    db = 'yes24',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 2023년 12월 25일 -> 2023-12-25
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

        match = re.search(f'(\d+)년 (\d+)월 (\d+)일', publishing)

        if match:
            year, month, day = match.group()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text

        reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",", ""))

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))
        
        try: 
            full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
            parts = full_text.split(" | ")
            ranking_part = parts[0]
            ranking = ''.join(filter(str.isdigit, ranking_part)) # 숫자만 추출
        except:
            ranking = 0

        try:
            # 국내도서 top20 3주
            ranking_weeks_part = parts[1]
            ranking_weeks= ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
        except:
            ranking_weeks = 0

        ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[1][:-1]
        ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]
     
    sql = """INSERT INTO Books(
        title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks) VALUES ($s, $s, $s, $s, $s, $s, $s, $s, $s
    )"""

    cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
    conn.commit()

    time.sleep(3)

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
/var/folders/fg/fv3ctzbs11g08lczrgk117y40000gn/T/ipykernel_55937/763986390.py:26: SyntaxWarning: invalid escape sequence '\d'
  match = re.search(f'(\d+)년 (\d+)월 (\d+)일', publishing)
/var/folders/fg/fv3ctzbs11g08lczrgk117y40000gn/T/ipykernel_55937/763986390.py:26: SyntaxWarning: invalid escape sequence '\d'
  match = re.search(f'(\d+)년 (\d+)월 (\d+)일', publishing)


NameError: name 'title' is not defined